<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP/blob/main/03%20Machine%20Learning/notebooks/P3-Practica-Clasificacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⭕ Parte I: Clasificación

Este conjunto de datos, proporcionado por Avazu, contiene registros de publicidad en línea (como impresiones de anuncios y clicks) recolectados durante 11 días. El objetivo es predecir la probabilidad de que un usuario haga clic en un anuncio (CTR), una métrica crítica para optimizar campañas de sponsored search y real-time bidding. La práctica consiste en desarrollar un modelo de clasificación para este fin, utilizando características como el tipo de dispositivo, contexto de la página y datos anónimos del usuario.

La descripción completa de las features se proporciona a continuación:

| Nombre Columna      | Tipo (Ejemplo)           | Descripción                                                                 | ¿Relevante para Modelo? |
|---------------------|--------------------------|-----------------------------------------------------------------------------|-------------------------|
| `id`               | String (ej: "100000")    | Identificador único del registro (anuncio).                                 | ❌ No                   |
| **`click`**        | **Binaria (0/1)**        | **Variable target**: 0 = No hubo clic, 1 = Sí hubo clic.                    | ✅ **Sí**               |
| `year`, `month`, `hour`             | Int        | Año, mes y hora           | ✅ Sí  |
| `C1`               | Categórica anónima       | Variable categórica encriptada (sin contexto definido).                     | ✅ Sí                   |
| `banner_pos`       | Numérica/Categórica      | Posición del banner en la página (ej: 0, 1, 2...).                          | ✅ Sí                   |
| `site_id`          | Categórica (ej: "1fbe01")| ID del sitio web donde se muestra el anuncio.                               | ✅ Sí (alta cardinalidad) |
| `site_domain`      | Categórica (ej: "f38457")| Dominio del sitio web.                                                      | ⚠️ Tal vez (redundante con `site_id`) |
| `site_category`    | Categórica (ej: "28905b")| Categoría del sitio (ej: noticias, deportes...).                            | ✅ Sí                   |
| `app_id`           | Categórica (ej: "ecadf6")| ID de la aplicación (si el anuncio se muestra en una app móvil).             | ✅ Sí (alta cardinalidad) |
| `app_domain`       | Categórica (ej: "7801e8")| Dominio de la aplicación.                                                   | ⚠️ Tal vez (redundante con `app_id`) |
| `app_category`     | Categórica (ej: "07d7df")| Categoría de la aplicación (ej: juegos, redes sociales...).                 | ✅ Sí                   |
| `device_id`        | Categórica (ej: "a99f2a")| ID del dispositivo del usuario.                                             | ⚠️ Alta cardinalidad (¿agrupar?) |
| `device_ip`        | Categórica (ej: "d1b8b4")| Dirección IP del dispositivo.                                               | ❌ No (a menos que se agrupe por región) |
| `device_model`     | Categórica (ej: "8a4875")| Modelo del dispositivo (ej: iPhone X, Samsung Galaxy S10...).               | ✅ Sí                   |
| `device_type`      | Categórica (ej: 1, 2)    | Tipo de dispositivo (ej: 1 = móvil, 2 = tablet, etc.).                      | ✅ Sí                   |
| `device_conn_type` | Categórica (ej: 0, 2)    | Tipo de conexión (ej: 0 = WiFi, 2 = 4G...).                                 | ✅ Sí                   |
| `C14` - `C21`      | Categóricas anónimas     | Variables categóricas encriptadas (sin contexto definido).                   | ✅ Sí (evaluar importancia) |

Dataset: https://www.kaggle.com/competitions/avazu-ctr-prediction/data

In [ ]:
import pandas as pd

url = 'https://github.com/DCDPUAEM/DCDP/raw/refs/heads/main/03%20Machine%20Learning/data/ctr_prediction.csv'

df = pd.read_csv(url)
df

## Instrucciones

0. Verifica si hay valores faltantes.
1. Extrae la variable target `click`. Verifica el balanceo de clases.
2. Quita las variables que consideres necesarias de acuerdo a la tabla anterior.
3. Forma la matriz de features `X`.
4. Convierte las variables categóricas en variables binarias mediante el one-hot encoding ([`get_dummies`](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html) de pandas). No olvides el hiperparámetro `drop_first=True` y `dtype=int`. Vas a obtener un dataframe con muchas columnas, es decir, ahora tenemos una muy alta dimensionalidad.
5. Divide en train/test
6. Verifica los rangos de las variables numéricas que tienes para la tarea de clasificación.
7. Realiza reducción de dimensionalidad con PCA con todas las componentes principales.
8. Escoge un número de componentes principales de manera que te quedes con el $\sim$80% de varianza.
9. Entrena un clasificador de tu elección en el conjunto de prueba. Evalua con la métrica F1-score
10. Reporta, además, la matriz de confusión.

# ⭕ Parte II: Segmentación y Recomendación

El dataset MovieLens 100K contiene información de 1682 películas, cada una representada por su título y 18 géneros cinematográficos (Action, Adventure, Drama, etc.), codificados como variables binarias. Estos datos, recolectados de calificaciones de 943 usuarios, permiten analizar las características intrínsecas de las películas independientemente de las preferencias de los usuarios.

El objetivo de esta práctica es realizar segmentación de películas mediante técnicas de clustering (como K-Means o clustering jerárquico) para identificar grupos naturales basados en sus géneros. Esto podría revelar patrones como *películas de acción con toques de sci-fi* o *dramas románticos con elementos musicales*, útiles para sistemas de recomendación o catálogos personalizados.

[Dataset completo en Kaggle](https://www.kaggle.com/datasets/prajitdatta/movielens-100k-dataset/data)

In [ ]:
import os
import pandas as pd
import kagglehub
import numpy as np

path = kagglehub.dataset_download("prajitdatta/movielens-100k-dataset")
path = '/root/.cache/kagglehub/datasets/prajitdatta/movielens-100k-dataset/versions/1/ml-100k'
fname = os.path.join(path, 'u.item')

movies_df = pd.read_csv(fname, sep='|', encoding='latin-1', names=[
    'movie_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL',
    'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime',
    'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
    'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'
])

def get_year(date):
    try:
        return int(date.split('-')[-1])
    except:
        return np.nan

movies_df.drop(['video_release_date', 'unknown','IMDb_URL','movie_id'], axis=1, inplace=True)
movies_df['date'] = movies_df['release_date'].apply(get_year)
movies_df.drop('release_date', axis=1, inplace=True)
movies_df

## Instrucciones

1. Verifica si hay variables faltantes.
2. Extraer las variables numéricas para formar la matriz de caracteristicas $X$.
3. Verifica los rangos de las variables, aplica re-escalamiento si lo consideras necesario.
4. Usa K-Means con algún valor $K$ de tu elección, haz clustering y mide el valor de silueta.
5. Encuentra el valor de codo y con ese valor, vuelve a hacer clustering y mide el valor de silueta.
6. Con el valor de $K$ que consideres mejor, extrae las etiquetas de los clusters y añadelas al dataframe `movies_df`.
7. Muestra un dataframe por cada cluster mostrando los títulos de las películas de cada cluster. ¿Es posible identificar, visualmente, qué define a cada cluster?
